## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-25-14-51-00 +0000,wsj,Stock Market Today: Dow and Dollar Rise; One W...,https://www.wsj.com/livecoverage/stock-market-...
1,2025-07-25-14-49-45 +0000,bbc,Former Congolese president on trial for treason,https://www.bbc.com/news/articles/cj4e4z892j5o
2,2025-07-25-14-47-00 +0000,wsj,"Week Ahead for FX, Bonds: Fed Decision, U.S. J...",https://www.wsj.com/economy/week-ahead-for-fx-...
3,2025-07-25-14-46-02 +0000,nypost,Iconic Philly cheesesteak shop may raise price...,https://nypost.com/2025/07/25/business/iconic-...
4,2025-07-25-14-44-45 +0000,nyt,The Generations of Pain I Felt in One Racist M...,https://www.nytimes.com/2025/07/25/us/racism-b...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,trump,60
153,gaza,14
941,he,13
96,epstein,12
90,court,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
170,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...,150
160,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,116
182,2025-07-24-23-54-32 +0000,nyt,Powell Tells Trump He Was Wrong About Renovati...,https://www.nytimes.com/2025/07/24/us/politics...,116
158,2025-07-25-03-00-00 +0000,wsj,As the U.S. heads toward trade talks with Chin...,https://www.wsj.com/economy/trade/trump-china-...,113
171,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...,110


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
170,150,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...
260,76,2025-07-24-19-57-36 +0000,cbc,"Israel, U.S. pull out of Gaza ceasefire talks ...",https://www.cbc.ca/news/world/france-macron-pa...
187,53,2025-07-24-23-50-13 +0000,nypost,Former top Biden White House official blames M...,https://nypost.com/2025/07/24/us-news/former-t...
171,50,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...
68,49,2025-07-25-11-05-54 +0000,nypost,Cat mom accuses Blue Angels of terrorizing her...,https://nypost.com/2025/07/25/us-news/cat-mom-...
216,44,2025-07-24-21-58-32 +0000,latimes,FCC approves Paramount-Skydance merger followi...,https://www.latimes.com/entertainment-arts/bus...
31,40,2025-07-25-13-46-47 +0000,nypost,Arizona woman who admitted to helping North Ko...,https://nypost.com/2025/07/25/us-news/arizona-...
65,39,2025-07-25-11-28-41 +0000,nypost,Sickos accused of running child sex-traffickin...,https://nypost.com/2025/07/25/us-news/sickos-a...
285,35,2025-07-24-18-31-01 +0000,nypost,Hulk Hogan became one of Trump’s biggest suppo...,https://nypost.com/2025/07/24/us-news/hulk-hog...
277,35,2025-07-24-19-08-13 +0000,bbc,King meets Modi after India-UK trade deal signed,https://www.bbc.com/news/articles/c307ggj492vo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
